# Installation & Setup for LangGraph with OpenSearch


In [1]:
## install langgraph-opensearch
# !pip install langgraph-opensearch==0.1.5

import os
os.environ["OSS_HOST"] = "xxx.xxx.xxx.xxx"
os.environ["OSS_PORT"] = "9200"

client_kwargs = {
    "hosts": [{"host": os.getenv('OSS_HOST'), "port": int(os.getenv('OSS_PORT'))}],
    "use_ssl": True,
    "http_auth": (os.environ.get("OSS_USER", "admin"), os.environ.get("OSS_PASS", "admin")),
    "verify_certs": False,
    "timeout": 30,
    "max_retries": 3,
}

import warnings
from urllib3.exceptions import InsecureRequestWarning
# Suppress the SSL warnings
warnings.filterwarnings("ignore", category=InsecureRequestWarning)

# Optionally suppress opensearch-py connection warnings
warnings.filterwarnings("ignore", category=UserWarning)


# Q&A with OpenSearch as Checkpoint Storage

In [2]:
from typing_extensions import TypedDict
from opensearchpy import OpenSearch
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.checkpoint.opensearch import OpenSearchSaver

from langchain_openai import ChatOpenAI

from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.0,
    max_tokens=1000,
    streaming=True,
)

def ask(state: MessagesState) -> MessagesState:
    """
    Ask a question to the LLM and return the response.
    """
    question = state['messages'][-1].content
    response = llm.invoke(
        [*state['messages']]
    )
    return { 'messages': [ AIMessage(content=response.content) ] }

graph = StateGraph(MessagesState)
graph.add_node('ask', ask)
graph.add_edge(START, 'ask')
graph.add_edge('ask', END)

# Initialize OpenSearchSaver with the graph
client = OpenSearch(**client_kwargs)
checkpointer = OpenSearchSaver(client=client)
config = {
    'configurable': {
        'thread_id': '1'
    }
}
graph = graph.compile(checkpointer=checkpointer)

response = graph.invoke(
    {
        "messages": [
            HumanMessage(content="If your AI agent crashes halfway through a complex task,\
            how can you restart it exactly from where it left off — \
            without re-running everything?")
        ]
    },
    config
)
print(response['messages'][-1].content)

response = graph.invoke(
    {
        "messages": [
            HumanMessage(content="And what if you not only resumed it, \
            but also explored alternative paths from that same point — \
            essentially giving your AI the ability to time travel?")
        ]
    },
    config
)
print(100*'*')
print(response['messages'][-1].content)


To restart an AI agent exactly from where it left off without re-running everything, you can implement a robust state-saving and checkpointing system. Here’s a detailed approach to achieve this:

1. **State Serialization**: Ensure that the AI agent's state, including its memory, variables, model parameters, and any relevant data, can be serialized. This involves converting the state into a format that can be saved to disk, such as using `pickle` in Python or similar serialization tools in other programming languages.

2. **Regular Checkpointing**: Implement a mechanism to periodically save the agent's state to disk. This can be done at regular intervals or after completing specific milestones within the task. The frequency of checkpointing should balance between minimizing data loss and avoiding excessive overhead.

3. **Incremental Saving**: Instead of saving the entire state every time, consider saving only the changes since the last checkpoint. This can reduce the amount of data tha

# Search Checkpoints

In [4]:
items = list(checkpointer.list(config))
print(items)

[CheckpointTuple(config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f076c03-90d2-6ad9-800a-5ea69dd93791'}}, checkpoint={'v': 4, 'ts': '2025-08-11T14:34:01.144802+00:00', 'id': '1f076c03-90d2-6ad9-800a-5ea69dd93791', 'channel_values': {'messages': [HumanMessage(content='If your AI agent crashes halfway through a complex task, how can you restart it exactly from where it left off — without re-running everything?', additional_kwargs={}, response_metadata={}, id='f91c6482-7753-4ff2-ba13-0e6bb3ad393c'), AIMessage(content="To restart an AI agent exactly from where it left off without re-running everything, you can implement a checkpointing and state-saving mechanism. Here are some steps to achieve this:\n\n1. **State Serialization**: Ensure that the AI agent's state, including its memory, variables, and any relevant data, can be serialized (converted into a format that can be saved to disk). This might involve using libraries like `pickle` in Python or similar

# Replay from a Checkpoint

In [5]:
to_replay = items[0].config
for event in graph.stream(None, to_replay, stream_mode="values"):
    if "messages" in event:
        print(f"Messages:{event['messages']}")

Messages:[HumanMessage(content='If your AI agent crashes halfway through a complex task, how can you restart it exactly from where it left off — without re-running everything?', additional_kwargs={}, response_metadata={}, id='f91c6482-7753-4ff2-ba13-0e6bb3ad393c'), AIMessage(content="To restart an AI agent exactly from where it left off without re-running everything, you can implement a checkpointing and state-saving mechanism. Here are some steps to achieve this:\n\n1. **State Serialization**: Ensure that the AI agent's state, including its memory, variables, and any relevant data, can be serialized (converted into a format that can be saved to disk). This might involve using libraries like `pickle` in Python or similar serialization tools in other languages.\n\n2. **Regular Checkpointing**: Periodically save the agent's state to disk at regular intervals or after completing specific milestones within the task. This is known as checkpointing. The frequency of checkpointing depends on 